# Common Practices for Spatial Data Science
* [code carbon tool](https://mlco2.github.io/codecarbon/)

In [19]:
from arcgis.features import GeoAccessor
import arcpy
from codecarbon import track_emissions
from configparser import ConfigParser
import logging
import numpy as np
import os
import pandas as pd

In [20]:
from traffic.read import read_sqlite_to_featureclass, read_sqlite_as_featureclass

## Ramp up the environment
* [Install and set up ArcGIS](https://developers.arcgis.com/python/guide/intro/)
* setup logging
* read config file
* fail early

In [21]:
def read_emissions(filepath: str):
    return pd.read_csv(filepath)

In [22]:
logging.basicConfig()
logger = logging.getLogger("codecarbon")
    
config = ConfigParser()
config.read("config.user")

traffic_filepath = config["DEFAULT"]["TrafficFilePath"]
if None is traffic_filepath:
    raise ValueError("Traffic file path not specified!")

## 1) Understanding where
If you don’t know where you are, you are lost. Understanding where is about
putting the world in context. Where are you? What is around you? Very similar
to when you were two years old, your journey of spatial analysis
requires an understanding of how you fit into your geography.

### The problem
The traffic data contains agent positions using latitude and longitude. We need to create geometries for mapping the agent positions.

### The solutions
* read the data using a [spatially enabled data frame](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/)

* read the data using a [insert cursor](https://pro.arcgis.com/en/pro-app/latest/arcpy/data-access/insertcursor-class.htm)

#### Two different geometry implementations
* [Geometry - ArcGIS API for Python](https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#geometry)
* [Geometry - arcpy](https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/geometry.htm)

In [23]:
@track_emissions(project_name="Urban Digital Twin Bonn - Read SDF", output_file="log/emissions-read.user", offline=True, country_iso_code="USA")
def track_read_sdf(traffic_filepath: str):
    read_sqlite_to_featureclass(traffic_filepath, "SELECT * FROM agent_pos;")
    
@track_emissions(project_name="Urban Digital Twin Bonn - Read FC", output_file="log/emissions-read.user", offline=True, country_iso_code="USA")
def track_read_fc(traffic_filepath: str):
    read_sqlite_as_featureclass(traffic_filepath, "SELECT * FROM agent_pos;")

In [24]:
track_read_sdf(traffic_filepath)

[codecarbon INFO @ 13:12:44] offline tracker init
[codecarbon INFO @ 13:12:44] [setup] RAM Tracking...
[codecarbon INFO @ 13:12:44] [setup] GPU Tracking...
[codecarbon INFO @ 13:12:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:12:44] [setup] CPU Tracking...
[codecarbon INFO @ 13:12:44] Tracking Intel CPU via Power Gadget
[codecarbon INFO @ 13:12:47] >>> Tracker's metadata:
[codecarbon INFO @ 13:12:47]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:12:47]   Python version: 3.11.8
[codecarbon INFO @ 13:12:47]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 13:12:47]   Available RAM : 31.775 GB
[codecarbon INFO @ 13:12:47]   CPU count: 24
[codecarbon INFO @ 13:12:47]   CPU model: 12th Gen Intel(R) Core(TM) i7-12800HX
[codecarbon INFO @ 13:12:47]   GPU count: 1
[codecarbon INFO @ 13:12:47]   GPU model: 1 x NVIDIA RTX A2000 8GB Laptop GPU
[codecarbon INFO @ 13:12:47] Saving emissions data to file C:\src\esride\green-spatial-engineering\notebooks\log\emissions-r

In [25]:
read_emissions_df = read_emissions("log/emissions-read.user")
read_emissions_df

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2024-09-13T13:08:54,Urban Digital Twin Bonn - Read SDF,5c2632a5-83ea-4868-bbbd-6102e94c28aa,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,24.702015,0.000067,0.000003,9.5245,121.831651,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
1,2024-09-13T13:09:23,Urban Digital Twin Bonn - Read FC,324e5469-8592-4930-92f3-86aa53aa0bd9,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.835404,0.000009,0.000002,11.0110,12.307493,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
2,2024-09-13T13:11:52,Urban Digital Twin Bonn - Read SDF,2afafbc5-e305-4d99-8ccb-199b78de9999,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.703971,0.000059,0.000003,9.6134,74.183817,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
3,2024-09-13T13:11:59,Urban Digital Twin Bonn - Read FC,b92e0e2e-a5fa-4ce9-bcdc-28d1935645ec,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.563481,0.000009,0.000002,9.2602,17.082928,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
4,2024-09-13T13:13:08,Urban Digital Twin Bonn - Read SDF,295dd0c6-eefe-459d-91d0-04b51affc264,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.629324,0.000065,0.000003,9.3615,99.305873,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0


In [26]:
track_read_fc(traffic_filepath)

[codecarbon INFO @ 13:13:08] offline tracker init
[codecarbon INFO @ 13:13:08] [setup] RAM Tracking...
[codecarbon INFO @ 13:13:08] [setup] GPU Tracking...
[codecarbon INFO @ 13:13:08] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:13:08] [setup] CPU Tracking...
[codecarbon INFO @ 13:13:08] Tracking Intel CPU via Power Gadget
[codecarbon INFO @ 13:13:10] >>> Tracker's metadata:
[codecarbon INFO @ 13:13:10]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:13:10]   Python version: 3.11.8
[codecarbon INFO @ 13:13:10]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 13:13:10]   Available RAM : 31.775 GB
[codecarbon INFO @ 13:13:11]   CPU count: 24
[codecarbon INFO @ 13:13:11]   CPU model: 12th Gen Intel(R) Core(TM) i7-12800HX
[codecarbon INFO @ 13:13:11]   GPU count: 1
[codecarbon INFO @ 13:13:11]   GPU model: 1 x NVIDIA RTX A2000 8GB Laptop GPU
[codecarbon INFO @ 13:13:11] Saving emissions data to file C:\src\esride\green-spatial-engineering\notebooks\log\emissions-r

In [27]:
read_emissions_df = read_emissions("log/emissions-read.user")
read_emissions_df

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2024-09-13T13:08:54,Urban Digital Twin Bonn - Read SDF,5c2632a5-83ea-4868-bbbd-6102e94c28aa,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,24.702015,0.000067,0.000003,9.5245,121.831651,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
1,2024-09-13T13:09:23,Urban Digital Twin Bonn - Read FC,324e5469-8592-4930-92f3-86aa53aa0bd9,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.835404,0.000009,0.000002,11.0110,12.307493,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
2,2024-09-13T13:11:52,Urban Digital Twin Bonn - Read SDF,2afafbc5-e305-4d99-8ccb-199b78de9999,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.703971,0.000059,0.000003,9.6134,74.183817,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
3,2024-09-13T13:11:59,Urban Digital Twin Bonn - Read FC,b92e0e2e-a5fa-4ce9-bcdc-28d1935645ec,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.563481,0.000009,0.000002,9.2602,17.082928,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
4,2024-09-13T13:13:08,Urban Digital Twin Bonn - Read SDF,295dd0c6-eefe-459d-91d0-04b51affc264,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.629324,0.000065,0.000003,9.3615,99.305873,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
5,2024-09-13T13:13:15,Urban Digital Twin Bonn - Read FC,ce922811-5428-45d9-a86f-817820f58823,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.660255,0.000009,0.000002,8.7851,14.794357,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0


### Deep dive

In [28]:
import pyinstrument

In [29]:
%load_ext pyinstrument

In [30]:
%%pyinstrument

traffic_featureclass = read_sqlite_to_featureclass(traffic_filepath, "SELECT * FROM agent_pos;")
traffic_featureclass

'memory\\traffic_data'

_     ._   __/__   _ _  _  _ _/_   Recorded: 13:13:25  Samples:  29926
 /_//_/// /_\ / //_// / //_'/ //     Duration: 31.652    CPU time: 18.469
/   _/                      v4.7.3

Profile at C:\Users\jts\AppData\Local\Temp\ipykernel_26060\1742315462.py:2

31.655 <cell line: 0>  ..\AppData\Local\Temp\ipykernel_26060\1742315462.py:1
`- 31.655 read_sqlite_to_featureclass  traffic\read.py:44
   |- 26.759 read_sqlite_as_sdf  traffic\read.py:37
   |  `- 26.499 from_xy  arcgis\features\geo\_accessor.py:3048
   |     `- 26.499 _from_xy  arcgis\features\geo\_io\fileops.py:178
   |        |- 14.144 Point.__getattribute__  arcgis\geometry\_types.py:191
   |        |  `- 14.034 Point.WKT  arcgis\geometry\_types.py:987
   |        |     |- 12.955 Point.__getattribute__  arcgis\geometry\_types.py:191
   |        |     |  `- 12.771 Point.as_arcpy  arcgis\geometry\_types.py:522
   |        |     |     |- 9.043 AsShape  arcpy\arcobjects\geometries.py:281
   |        |     |     |  |- 7.322 Geoprocessor.fromEsriJson  arcpy\geoprocessing\_base.py:482
   |        |     |     |  |  |- 6.693 geoprocessing object.fromesrijson  <built-in>
   |        |     |     |  |  `- 0.629 [self]  arcpy\geoprocessing\_base.py
   |        |     |     |  |- 1.348 convertArcObjectToPythonObject  arcpy\arcobjects\arcobjectconversion.py:59
   |        |     |     |  |  |- 0.810 Geometry.FromScriptingArcObject  arcpy\arcobjects\_base.py:178
   |        |     |     |  |  |  `- 0.696 Geometry.__from_scripting_arc_object__  arcpy\arcobjects\mixins.py:99
   |        |     |     |  |  |     `- 0.668 [self]  arcpy\arcobjects\mixins.py
   |        |     |     |  |  `- 0.509 [self]  arcpy\arcobjects\arcobjectconversion.py
   |        |     |     |  `- 0.365 [self]  arcpy\arcobjects\geometries.py
   |        |     |     |- 1.635 dumps  json\__init__.py:183
   |        |     |     |  `- 1.483 JSONEncoder.encode  json\encoder.py:183
   |        |     |     |     `- 1.234 JSONEncoder.iterencode  json\encoder.py:205
   |        |     |     |        `- 1.119 [self]  json\encoder.py
   |        |     |     `- 1.169 [self]  arcgis\geometry\_types.py
   |        |     `- 0.752 PointGeometry._get  arcpy\arcobjects\_base.py:87
   |        |        `- 0.577 getattr  <built-in>
   |        |- 10.048 _iLocIndexer.__getitem__  pandas\core\indexing.py:1089
   |        |  `- 9.744 _iLocIndexer._getitem_axis  pandas\core\indexing.py:1623
   |        |     |- 8.343 DataFrame._ixs  pandas\core\frame.py:3639
   |        |     |  |- 5.772 BlockManager.fast_xs  pandas\core\internals\managers.py:1027
   |        |     |  |  |- 1.866 interleaved_dtype  pandas\core\internals\base.py:208
   |        |     |  |  |  `- 1.787 find_common_type  pandas\core\dtypes\cast.py:1414
   |        |     |  |  |     |- 0.718 [self]  pandas\core\dtypes\cast.py
   |        |     |  |  |     `- 0.410 GeoType._get_common_dtype  pandas\core\dtypes\base.py:358
   |        |     |  |  |- 1.641 [self]  pandas\core\internals\managers.py
   |        |     |  |  |- 0.661 new_block  pandas\core\internals\blocks.py:2385
   |        |     |  |  `- 0.472 ExtensionBlock.iget  pandas\core\internals\blocks.py:1879
   |        |     |  |- 1.312 Series.__init__  pandas\core\series.py:368
   |        |     |  |  `- 0.892 Series.__setattr__  pandas\core\generic.py:5991
   |        |     |  |     |- 0.365 Series.name  pandas\core\series.py:671
   |        |     |  |     `- 0.333 Series.name  pandas\core\series.py:621
   |        |     |  |- 0.452 Series.__finalize__  pandas\core\generic.py:5931
   |        |     |  `- 0.374 [self]  pandas\core\frame.py
   |        |     |- 0.479 is_bool_indexer  pandas\core\common.py:95
   |        |     `- 0.369 _iLocIndexer._validate_integer  pandas\core\indexing.py:1571
   |        |- 1.288 Series.__getitem__  pandas\core\series.py:992
   |        |  `- 0.804 Series._get_value  pandas\core\series.py:1099
   |        |     `- 0.335 Series._values  pandas\core\series.py:718
   |        |- 0.484 vectorize.

In [31]:
traffic_sdf = GeoAccessor.from_featureclass(traffic_featureclass)
traffic_sdf

,OBJECTID,id,trip,person,vehicle_type,distance_crossed,latitude,longitude,trip_time,SHAPE
0,1,1,34991,21889,Car,59186982,50.722015,7.100067,2023-10-16T07:50:00,"{""x"": 7.10006697700004, ""y"": 50.72201472400007..."
1,2,2,53482,33713,Car,59897895,50.722031,7.099976,2023-10-16T07:50:00,"{""x"": 7.099976309000056, ""y"": 50.7220306040000..."
2,3,3,32417,20277,Car,38111791,50.722047,7.099882,2023-10-16T07:50:00,"{""x"": 7.0998823140000695, ""y"": 50.722047067000..."
3,4,4,53609,33840,Car,48612537,50.722064,7.099784,2023-10-16T07:50:00,"{""x"": 7.099783692000074, ""y"": 50.7220643400000..."
4,5,5,46191,28872,Car,54200035,50.722081,7.099691,2023-10-16T07:50:00,"{""x"": 7.0996909710000295, ""y"": 50.722080580000..."
...,...,...,...,...,...,...,...,...,...,...
62332,62259,62259,51635,32300,,54021983,50.739271,7.100041,2023-10-16T07:51:55,"{""x"": 7.100040705000026, ""y"": 50.7392705200000..."
62333,62260,62260,51720,32350,,1371583,50.730012,7.096168,2023-10-16T07:51:55,"{""x"": 7.096167971000057, ""y"": 50.7300116140000..."
62334,62261,62261,51788,32391,,8227092,50.723101,7.138357,2023-10-16T07:51:55,"{""x"": 7.138356961000056, ""y"": 50.7231014720000..."
62335,62262,62262,51893,32461,,2814269,50.735459,7.101884,2023-10-16T07:51:55,"{""x"": 7.101884386000052, ""y"": 50.7354587150000..."


In [32]:
%%pyinstrument

traffic_featureclass = read_sqlite_as_featureclass(traffic_filepath, "SELECT * FROM agent_pos;")
traffic_featureclass

'memory\\traffic_data'

_     ._   __/__   _ _  _  _ _/_   Recorded: 13:14:11  Samples:  2570
 /_//_/// /_\ / //_// / //_'/ //     Duration: 3.381     CPU time: 2.047
/   _/                      v4.7.3

Profile at C:\Users\jts\AppData\Local\Temp\ipykernel_26060\1976316551.py:2

3.380 <cell line: 0>  ..\AppData\Local\Temp\ipykernel_26060\1976316551.py:1
`- 3.380 read_sqlite_as_featureclass  traffic\read.py:84
   |- 3.115 _read_traffic_as_featureclass  traffic\read.py:94
   |  |- 1.243 record.__getitem__  numpy\core\records.py:281
   |  |  |- 1.125 [self]  numpy\core\records.py
   |  |  `- 0.118 isinstance  <built-in>
   |  |- 0.605 [self]  traffic\read.py
   |  |- 0.560 InsertCursor.insertRow  <built-in>
   |  |- 0.214 CreateFeatureclass  arcpy\management.py:3547
   |  |  `- 0.214 <lambda>  arcpy\geoprocessing\_base.py:512
   |  |- 0.146 AddFields  arcpy\management.py:5270
   |  |  `- 0.146 <lambda>  arcpy\geoprocessing\_base.py:512
   |  |- 0.143 recarray.__getitem__  numpy\core\records.py:508
   |  |  |- 0.096 [self]  numpy\core\records.py
   |  |  `- 0.040 record.__getattribute__  numpy\core\records.py:242
   |  |- 0.134 ClearWorkspaceCache  arcpy\management.py:23406
   |  |  `- 0.134 <lambda>  arcpy\geoprocessing\_base.py:512
   |  |- 0.035 DataFrame.to_records  pandas\core\frame.py:2321
   |  |  `- 0.034 fromarrays  numpy\core\records.py:588
   |  `- 0.035 InsertCursor.__exit__  <built-in>
   `- 0.265 read_sqlite_as_df  traffic\read.py:30
      `- 0.265 read_sql_query  pandas\io\sql.py:386
         `- 0.260 SQLiteDatabase.read_query  pandas\io\sql.py:2255
            |- 0.174 SQLiteDatabase._fetchall_as_list  pandas\io\sql.py:2295
            |  `- 0.174 Cursor.fetchall  <built-in>
            `- 0.085 _wrap_result  pandas\io\sql.py:168
               `- 0.082 _convert_arrays_to_dataframe  pandas\io\sql.py:142
                  `- 0.058 convert_object_array  pandas\core\internals\construction.py:1001
                     `- 0.058 <listcomp>  pandas\core\internals\construction.py:1067
                        `- 0.058 convert  pandas\core\internals\construction.py:1023
                           `- 0.058 maybe_convert_objects  <built-in>

In [33]:
traffic_sdf = GeoAccessor.from_featureclass(traffic_featureclass)
traffic_sdf

,OBJECTID,id,trip,person,vehicle_type,distance_crossed,longitude,latitude,trip_time,SHAPE
0,1,1,34991,21889,Car,59186982,50.722015,7.100067,2023-10-16 07:50:00,"{""x"": 50.722014724000076, ""y"": 7.1000669770000..."
1,2,2,53482,33713,Car,59897895,50.722031,7.099976,2023-10-16 07:50:00,"{""x"": 50.722030604000054, ""y"": 7.0999763090000..."
2,3,3,32417,20277,Car,38111791,50.722047,7.099882,2023-10-16 07:50:00,"{""x"": 50.72204706700006, ""y"": 7.09988231400006..."
3,4,4,53609,33840,Car,48612537,50.722064,7.099784,2023-10-16 07:50:00,"{""x"": 50.72206434000003, ""y"": 7.09978369200007..."
4,5,5,46191,28872,Car,54200035,50.722081,7.099691,2023-10-16 07:50:00,"{""x"": 50.72208058000007, ""y"": 7.09969097100002..."
...,...,...,...,...,...,...,...,...,...,...
62332,62259,62259,51635,32300,<NA>,54021983,50.739271,7.100041,2023-10-16 07:51:55,"{""x"": 50.73927052000005, ""y"": 7.10004070500002..."
62333,62260,62260,51720,32350,<NA>,1371583,50.730012,7.096168,2023-10-16 07:51:55,"{""x"": 50.730011614000034, ""y"": 7.0961679710000..."
62334,62261,62261,51788,32391,<NA>,8227092,50.723101,7.138357,2023-10-16 07:51:55,"{""x"": 50.72310147200005, ""y"": 7.13835696100005..."
62335,62262,62262,51893,32461,<NA>,2814269,50.735459,7.101884,2023-10-16 07:51:55,"{""x"": 50.73545871500005, ""y"": 7.10188438600005..."


### The trade offs and alternatives

#### Use numpy to feature class
Your tools of choice are *df.apply*, *df.transform* and *dtypes*

[NumPy in ArcGIS](https://pro.arcgis.com/en/pro-app/3.2/arcpy/get-started/working-with-numpy-in-arcgis.htm)

We need to construct a complex numpy array.

In [34]:
traffic_arr = np.array([(1, (471316.3835861763, 5000448.782036674)), 
          (2, (470402.49348005146, 5000049.216449278))], np.dtype([('idfield', np.int32),('XY', '<f8', 2)]))
traffic_arr

array([(1, [ 471316.38358618, 5000448.78203667]),
       (2, [ 470402.49348005, 5000049.21644928])],
      dtype=[('idfield', '<i4'), ('XY', '<f8', (2,))])

We need to necessary modules for converting numpy arrays

In [35]:
from arcpy.da import NumPyArrayToFeatureClass
from arcpy.management import Delete

In [36]:
def df_to_numpy(traffic_df: pd.DataFrame):
    return np.array([(trip, (longitude, latitude),) for trip, longitude, latitude in traffic_df[['trip', 'longitude', 'latitude']].values], 
                       np.dtype([('idfield', np.int32),('XY', '<f8', 2)]))

def numpy_to_featureclass(array, out_featureclass: str):
    if arcpy.Exists(out_featureclass):
        Delete(out_featureclass)
    
    NumPyArrayToFeatureClass(traffic_arr, out_featureclass, ['XY'])
    return out_featureclass

In [37]:
traffic_featureclass_arr = numpy_to_featureclass(traffic_arr, 'memory/traffic_data_arr')
GeoAccessor.from_featureclass(traffic_featureclass_arr)

,OBJECTID,idfield,SHAPE
0,1,1,"{""x"": 471316.38372802734, ""y"": 5000448.7821044..."
1,2,2,"{""x"": 470402.49353027344, ""y"": 5000049.2164916..."


In [38]:
traffic_arr = df_to_numpy(traffic_sdf)
traffic_arr

array([(34991, [50.72201472,  7.10006698]),
       (53482, [50.7220306 ,  7.09997631]),
       (32417, [50.72204707,  7.09988231]), ...,
       (51788, [50.72310147,  7.13835696]),
       (51893, [50.73545871,  7.10188439]),
       (51948, [50.70948401,  7.10938861])],
      dtype=[('idfield', '<i4'), ('XY', '<f8', (2,))])

In [39]:
traffic_featureclass_arr = numpy_to_featureclass(traffic_arr, 'memory/traffic_data_arr')
GeoAccessor.from_featureclass(traffic_featureclass_arr)

,OBJECTID,idfield,SHAPE
0,1,34991,"{""x"": 50.72210693359375, ""y"": 7.10009765625, ""..."
1,2,53482,"{""x"": 50.72210693359375, ""y"": 7.10009765625, ""..."
2,3,32417,"{""x"": 50.72210693359375, ""y"": 7.09991455078125..."
3,4,53609,"{""x"": 50.72210693359375, ""y"": 7.09991455078125..."
4,5,46191,"{""x"": 50.72210693359375, ""y"": 7.09967041015625..."
...,...,...,...
62332,62259,45064,"{""x"": 50.72271728515625, ""y"": 7.096923828125, ..."
62333,62260,45102,"{""x"": 50.7401123046875, ""y"": 7.12188720703125,..."
62334,62261,45187,"{""x"": 50.718505859375, ""y"": 7.0718994140625, ""..."
62335,62262,45400,"{""x"": 50.72308349609375, ""y"": 7.07830810546875..."


In [40]:
from traffic.read import read_sqlite_as_df

In [41]:
%%pyinstrument

traffic_df = read_sqlite_as_df(traffic_filepath, "SELECT * FROM agent_pos;")
traffic_arr = df_to_numpy(traffic_df)
traffic_featureclass_arr = numpy_to_featureclass(traffic_arr, 'memory/traffic_data_arr')
traffic_featureclass_arr

'memory/traffic_data_arr'

_     ._   __/__   _ _  _  _ _/_   Recorded: 13:14:18  Samples:  27
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.782     CPU time: 0.391
/   _/                      v4.7.3

Profile at C:\Users\jts\AppData\Local\Temp\ipykernel_26060\3935193772.py:2

0.789 <cell line: 0>  ..\AppData\Local\Temp\ipykernel_26060\3935193772.py:1
|- 0.410 numpy_to_featureclass  ..\AppData\Local\Temp\ipykernel_26060\3507771954.py:5
|  |- 0.228 NumPyArrayToFeatureClass  <built-in>
|  `- 0.179 Delete  arcpy\management.py:7629
|     `- 0.179 <lambda>  arcpy\geoprocessing\_base.py:512
|- 0.258 read_sqlite_as_df  traffic\read.py:30
|  `- 0.258 read_sql_query  pandas\io\sql.py:386
|     `- 0.253 SQLiteDatabase.read_query  pandas\io\sql.py:2255
|        |- 0.169 SQLiteDatabase._fetchall_as_list  pandas\io\sql.py:2295
|        |  `- 0.169 Cursor.fetchall  <built-in>
|        `- 0.084 _wrap_result  pandas\io\sql.py:168
|           `- 0.081 _convert_arrays_to_dataframe  pandas\io\sql.py:142
|              |- 0.056 convert_object_array  pandas\core\internals\construction.py:1001
|              |  `- 0.056 <listcomp>  pandas\core\internals\construction.py:1067
|              |     `- 0.056 convert  pandas\core\internals\construction.py:1023
|              |        `- 0.056 maybe_convert_objects  <built-in>
|              |- 0.013 DataFrame.__init__  pandas\core\frame.py:641
|              |  `- 0.013 dict_to_mgr  pandas\core\internals\construction.py:411
|              |     `- 0.013 arrays_to_mgr  pandas\core\internals\construction.py:97
|              |        `- 0.011 create_block_manager_from_column_arrays  pandas\core\internals\managers.py:2119
|              `- 0.012 to_object_array_tuples  <built-in>
`- 0.120 df_to_numpy  ..\AppData\Local\Temp\ipykernel_26060\3507771954.py:1
   |- 0.076 <listcomp>  ..\AppData\Local\Temp\ipykernel_26060\3507771954.py:2
   `- 0.037 array  <built-in>

In [42]:
@track_emissions(project_name="Urban Digital Twin Bonn - Read NP", output_file="log/emissions-read.user", offline=True, country_iso_code="USA")
def track_read_np(traffic_filepath: str):
    traffic_df = read_sqlite_as_df(traffic_filepath, "SELECT * FROM agent_pos;")
    traffic_arr = df_to_numpy(traffic_df)
    numpy_to_featureclass(traffic_arr, 'memory/traffic_data_arr')

In [43]:
track_read_np(traffic_filepath)

[codecarbon INFO @ 13:14:19] offline tracker init
[codecarbon INFO @ 13:14:19] [setup] RAM Tracking...
[codecarbon INFO @ 13:14:19] [setup] GPU Tracking...
[codecarbon INFO @ 13:14:19] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:14:19] [setup] CPU Tracking...
[codecarbon INFO @ 13:14:19] Tracking Intel CPU via Power Gadget
[codecarbon INFO @ 13:14:22] >>> Tracker's metadata:
[codecarbon INFO @ 13:14:22]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:14:22]   Python version: 3.11.8
[codecarbon INFO @ 13:14:22]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 13:14:22]   Available RAM : 31.775 GB
[codecarbon INFO @ 13:14:22]   CPU count: 24
[codecarbon INFO @ 13:14:22]   CPU model: 12th Gen Intel(R) Core(TM) i7-12800HX
[codecarbon INFO @ 13:14:22]   GPU count: 1
[codecarbon INFO @ 13:14:22]   GPU model: 1 x NVIDIA RTX A2000 8GB Laptop GPU
[codecarbon INFO @ 13:14:22] Saving emissions data to file C:\src\esride\green-spatial-engineering\notebooks\log\emissions-r

In [44]:
read_emissions_df = read_emissions("log/emissions-read.user")
read_emissions_df.sort_values(by='emissions')

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
6,2024-09-13T13:14:25,Urban Digital Twin Bonn - Read NP,88480f11-56f8-40e4-b9b3-4f8186f03965,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,3.058111,0.000005,0.000002,9.4453,39.980806,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
5,2024-09-13T13:13:15,Urban Digital Twin Bonn - Read FC,ce922811-5428-45d9-a86f-817820f58823,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.660255,0.000009,0.000002,8.7851,14.794357,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
1,2024-09-13T13:09:23,Urban Digital Twin Bonn - Read FC,324e5469-8592-4930-92f3-86aa53aa0bd9,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.835404,0.000009,0.000002,11.0110,12.307493,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
3,2024-09-13T13:11:59,Urban Digital Twin Bonn - Read FC,b92e0e2e-a5fa-4ce9-bcdc-28d1935645ec,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.563481,0.000009,0.000002,9.2602,17.082928,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
2,2024-09-13T13:11:52,Urban Digital Twin Bonn - Read SDF,2afafbc5-e305-4d99-8ccb-199b78de9999,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.703971,0.000059,0.000003,9.6134,74.183817,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
4,2024-09-13T13:13:08,Urban Digital Twin Bonn - Read SDF,295dd0c6-eefe-459d-91d0-04b51affc264,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.629324,0.000065,0.000003,9.3615,99.305873,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
0,2024-09-13T13:08:54,Urban Digital Twin Bonn - Read SDF,5c2632a5-83ea-4868-bbbd-6102e94c28aa,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,24.702015,0.000067,0.000003,9.5245,121.831651,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0


In [45]:
import geopandas as gpd
from geopandas import GeoDataFrame

In [46]:
def df_to_gpd(traffic_df: pd.DataFrame):
    return GeoDataFrame(traffic_df, geometry=gpd.points_from_xy(traffic_df['longitude'], traffic_df['latitude'], crs='EPSG:4326'))

In [47]:
@track_emissions(project_name="Urban Digital Twin Bonn - Read GPD", output_file="log/emissions-read.user", offline=True, country_iso_code="USA")
def track_read_gpd(traffic_filepath: str):
    traffic_df = read_sqlite_as_df(traffic_filepath, "SELECT * FROM agent_pos;")
    traffic_gpd = df_to_gpd(traffic_df)
    GeoAccessor.from_geodataframe(traffic_gpd, inplace=True, column_name='SHAPE')
    numpy_to_featureclass(traffic_arr, 'memory/traffic_data_arr')

In [48]:
track_read_gpd(traffic_filepath)

[codecarbon INFO @ 13:14:26] offline tracker init
[codecarbon INFO @ 13:14:26] [setup] RAM Tracking...
[codecarbon INFO @ 13:14:26] [setup] GPU Tracking...
[codecarbon INFO @ 13:14:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:14:26] [setup] CPU Tracking...
[codecarbon INFO @ 13:14:26] Tracking Intel CPU via Power Gadget
[codecarbon INFO @ 13:14:29] >>> Tracker's metadata:
[codecarbon INFO @ 13:14:29]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:14:29]   Python version: 3.11.8
[codecarbon INFO @ 13:14:29]   CodeCarbon version: 2.7.1
[codecarbon INFO @ 13:14:29]   Available RAM : 31.775 GB
[codecarbon INFO @ 13:14:29]   CPU count: 24
[codecarbon INFO @ 13:14:29]   CPU model: 12th Gen Intel(R) Core(TM) i7-12800HX
[codecarbon INFO @ 13:14:29]   GPU count: 1
[codecarbon INFO @ 13:14:29]   GPU model: 1 x NVIDIA RTX A2000 8GB Laptop GPU
[codecarbon INFO @ 13:14:29] Saving emissions data to file C:\src\esride\green-spatial-engineering\notebooks\log\emissions-r

In [49]:
read_emissions_df = read_emissions("log/emissions-read.user")
read_emissions_df.sort_values(by='emissions')

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
6,2024-09-13T13:14:25,Urban Digital Twin Bonn - Read NP,88480f11-56f8-40e4-b9b3-4f8186f03965,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,3.058111,0.000005,0.000002,9.4453,39.980806,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
7,2024-09-13T13:14:34,Urban Digital Twin Bonn - Read GPD,d20c1b5c-c237-4dea-b665-1609eb925bdc,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.853276,0.000009,0.000002,9.8763,11.230619,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
5,2024-09-13T13:13:15,Urban Digital Twin Bonn - Read FC,ce922811-5428-45d9-a86f-817820f58823,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.660255,0.000009,0.000002,8.7851,14.794357,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
1,2024-09-13T13:09:23,Urban Digital Twin Bonn - Read FC,324e5469-8592-4930-92f3-86aa53aa0bd9,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.835404,0.000009,0.000002,11.0110,12.307493,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
3,2024-09-13T13:11:59,Urban Digital Twin Bonn - Read FC,b92e0e2e-a5fa-4ce9-bcdc-28d1935645ec,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,4.563481,0.000009,0.000002,9.2602,17.082928,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
2,2024-09-13T13:11:52,Urban Digital Twin Bonn - Read SDF,2afafbc5-e305-4d99-8ccb-199b78de9999,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.703971,0.000059,0.000003,9.6134,74.183817,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
4,2024-09-13T13:13:08,Urban Digital Twin Bonn - Read SDF,295dd0c6-eefe-459d-91d0-04b51affc264,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,20.629324,0.000065,0.000003,9.3615,99.305873,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
0,2024-09-13T13:08:54,Urban Digital Twin Bonn - Read SDF,5c2632a5-83ea-4868-bbbd-6102e94c28aa,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,24.702015,0.000067,0.000003,9.5245,121.831651,11.915723,...,24,12th Gen Intel(R) Core(TM) i7-12800HX,1,1 x NVIDIA RTX A2000 8GB Laptop GPU,NaN,NaN,31.775261,machine,N,1.0
